## Paths, packages, and parameters

In [2]:
import pandas as pd
import numpy as np
import os
import cv2 as cv  # openCV for interacting with video
from matplotlib import pyplot as plt
# import videoTools as vt

# Change to for each computer
root_path = '/Users/mmchenry/Documents/Projects/geotaxis'

# Subdirectories
csv_path  = root_path + '/Seastar geotaxis experiment log - Experiments.csv'
im_path   = root_path + '/Raw_images'
out_path  = root_path + '/Videos'

# Suffix for images
suffix = "JPG"

# Parameters for selecting ROI
show_crosshair = True
from_center=True

## Convert image sequences, based on spreadsheet entries

In [4]:
# Input & output frame rates
# These need to be the same or ffmpeg will add or skip frames
fps_in = 1
fps_out = fps_in

# Image quality (0 to 1) for output video
imQuality = 0.75

# Quality value, on the 51-point scale used by ffmpeg
qVal = 51 * (1 - imQuality)

# Open CSV file
file = open(csv_path)

# Import CSV data
d = pd.read_csv(file)

# Number of videos to analyze
nVids = np.nansum(d.make_video)

# COunter for number of videos analyzed
k = 1

# Loop though each row 
for n in range(len(d)):

    if d.make_video[n]==1:

        print('Converting ' + str(d.date[n]) + '_' + str(int(d.trial_num[n])) + ' . . .')

        # Read number of frames from spreadsheet
        frStart = int(d.start_image_filename[n][3:])
        frEnd   = int(d.end_image_filename[n][3:])
        nFrames = frEnd - frStart

        print('     ' + str(nFrames) + ' frames, starting with ' + str(d.start_image_filename[n]) + '.JPG' )

        vidPath = out_path + '/' + str(d.date[n]) + '_' + str(int(d.trial_num[n])) + '.mp4'
        imagePath = im_path + '/' + str(d.date[n]) + '/'

        # Check that roi is defined
        if np.isnan(d.roi_x[n]):
            raise Exception("You need to define the region-of-interest")

        # Define ROI
        r = [int(d.roi_x[n]), int(d.roi_y[n]), int(d.roi_w[n]), int(d.roi_h[n])]

        # Excute ffmpeg
        command = f"ffmpeg -framerate {fps_in} -start_number {frStart}  -loglevel warning -i {imagePath}/DSC%05d.{suffix} " \
                  f"-vframes {nFrames} -y -vcodec libx264 -pix_fmt yuv420p -an -r {fps_out} -crf {qVal} -vf " \
                  f"\"crop= {r[2]}:{r[3]}:{r[0]}:{r[1]}\" " \
                  f"-timecode 00:00:00:00  '{vidPath}'"
        os.system(command)

        print('     Completed ' + str(int(k)) + ' of ' + str(int(nVids)) + ' videos')
        # print('Completed videos' + )
        k = k + 1

# Close CSV
file.close()


Converting 2022-01-25_2 . . .
     125 frames, starting with DSC01024.JPG


Exception: You need to define the region-of-interest